# CNN

- 필터를 적용할때 이미지 왼쪽 위에서 오른쪽 밑까지 밀어가며 곱하고 더하는 작업을 Convolution 이라고 함
- CNN 은 이미지를 추출하는 필터를 학습
- CNN 은 Convolution layer 와 Pooling Layer 로 구성되어 있음
    - Convolution Layer: 이미지의 특징을 추출하는 역할
    - Pooling Layer: 필터를 거친 여러 특징 중 가장 중요한 특징 하나를 골라냄
        - 덜 중요한 특징을 버리기 때문에 이미지의 차원이 감소
- CNN 은 사물이 조금만 치우쳐져도 인식하지 못하던 인공 신경망의 문제를 이미지 전체에 필터를 적용해 특징을 추출하는 방식으로 해결해줌
    - 이미지 크기만큼의 가중치를 가져야 하는 일반 인공 신경망과는 다르게 필터만을 학습시키면 되어 훨씬 적은 계산량으로 효율적인 학습이 가능 

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
EPOCHS = 10 # 원래 40 이지만 시간을 위해 10으로 변경
BATCH_SIZE = 64

#### 데이터셋 불러오기

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                   train=True,
                   download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                   train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)

#### 뉴럴넷으로 Fashion MNIST 학습하기

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # nn.Conv2d 의 두 파라미터는 입력 채널 수와 출력 채널 수 
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # kernel 의 크기는 5x5, 흑백이미지이므로 색상채널은 1
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return x

In [6]:
model = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

#### 학습하기

In [7]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

#### 테스트하기

In [8]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 배치 오차를 합산
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()

            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [9]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.316996
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.239870
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.840894
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.519204
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.529822
[1] Test Loss: 0.1927, Accuracy: 94.18%
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.275453
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.251754
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.413641
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.275830
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.185384
[2] Test Loss: 0.1289, Accuracy: 96.00%
Train Epoch: 3 [0/60000 (0%)]	Loss: 0.316581
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.583790
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.220154
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.273947
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.568021
[3] Test Loss: 0.0970, Accuracy: 97.01%
Train Epoch: 4 [0/60000 (0%)]	Loss: 0.245578
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.258500
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.300114


# ResNet

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets, models

In [11]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [12]:
EPOCHS = 300
BATCH_SIZE = 128

In [13]:
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                   train=True,
                   download=True,
                   transform=transforms.Compose([
                       transforms.RandomCrop(32, padding=4),
                       transforms.RandomHorizontalFlip(),
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5),
                                            (0.5, 0.5, 0.5))])),
    batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./.data',
                   train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5),
                                            (0.5, 0.5, 0.5))])),
    batch_size=BATCH_SIZE, shuffle=True)

100%|████████████████████████| 170498071/170498071 [00:19<00:00, 8942373.39it/s]


Extracting ./.data/cifar-10-python.tar.gz to ./.data


- 여러 단계의 신경망을 거치며 최초 입력 이미지에 대한 정보가 소실되기 때문에 무작정 신경망을 여러 개 겹친다고 학습 성능이 무한히 좋아지지 않는다
- ResNet 의 핵심은 네트워크를 작은 블록인 Residual 블록으로 나눈 것
    - Residual 블록의 출력에 입력이였던 x 를 더함으로써 모델을 더 깊게 설계 가능 
        - 입력과 출력의 관계를 바로 학습하기보다 입력과 출력의 차이를 따로 학습하는게 성능이 좋다는 가설

## ResNet 모델 만들기

In [ ]:
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        
        
        
        
    